In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Flatten,Conv2D,MaxPool2D,Dropout,BatchNormalization,Dense
from keras.preprocessing.text import one_hot
from keras.utils.np_utils import to_categorical   
from PIL import Image

import tensorflow as tf
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv("/kaggle/input/sign-language-mnist/sign_mnist_train.csv")

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
y=df.label.values
x=df.iloc[:,1:].values

In [ ]:
x.shape

In [ ]:
#reshaphing the data for ploting
x=x.reshape(x.shape[0],28,28)

In [ ]:
from skimage import io
io.imshow(x[2])

In [ ]:
fig, axes = plt.subplots(10,10, figsize=(8,8))
for i,ax in enumerate(axes.flat):
    ax.imshow(x[i])

In [ ]:
#spliting the data for training and testing

x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=1)

In [ ]:
#normalization 
x_train=x_train/255
x_test=x_test/255

In [ ]:
x_train=x_train.reshape(x_train.shape[0],28,28,1)
x_test=x_test.reshape(x_test.shape[0],28,28,1)

In [ ]:

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
#tpu configuration

tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)


In [ ]:
with tpu_strategy.scope():
    model=Sequential()
    model.add(Conv2D(50,(5,5),activation='relu',padding='same',strides=1,input_shape=(28,28,1)))
    model.add(Conv2D(30,(5,5),activation='relu',padding='same',strides=1))
    model.add(MaxPool2D((2,2)))
    model.add(Conv2D(50,(5,5),activation='relu',padding='same',strides=1))
    model.add(Conv2D(50,(5,5),activation='relu',padding='same',strides=1))
    model.add(MaxPool2D((2,2)))
    model.add(Dropout(.20))
    model.add(Flatten())
    model.add(Dense(25,activation='softmax'))
    
    model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history=model.fit(x_train,y_train,epochs=10,validation_data=(x_test,y_test),batch_size=50)

In [ ]:
#ploting the model accuracy and loss

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()